In [2]:
import pymrio
exio3 = pymrio.parse_exiobase3(path="exiobase/IOT_2022_pxp.zip")
import country_converter as coco
cc = coco.CountryConverter()
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Suppress future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
region_agg = coco.agg_conc(original_countries='EXIO3', aggregates=["EU"], missing_countries='Non EU', merge_multiple_string=None)
exio3.aggregate(region_agg = region_agg)
exio3.calc_all()
exio3_ori = exio3.copy()

In [4]:
vehicles_list = ["ICEV","PHEV","BEV","FCEV"]
scenario_list = ["EU0","EU100"]
#scenario = "EU0"
desired_order = exio3.get_sectors()
stressor = "Carbon dioxide (CO2) IPCC categories 1 to 4 and 6 to 7 (excl land use, land use change and forestry)"

In [5]:
file_path = "inputs/input_manufacturing.csv"
requirement = pd.read_csv(file_path)
requirement = pd.melt(requirement, id_vars=['Product'], value_vars=['ICEV', 'PHEV', 'BEV', 'FCEV'],
                    var_name='vehicle', value_name='requirement')
requirement.rename(columns={'Product': 'sector'},inplace=True)
requirement_product = requirement.pivot_table(index=['sector'],columns='vehicle',values='requirement',aggfunc='sum').reindex(index=desired_order)
requirement_product.fillna(0,inplace=True)

In [6]:
requirement_product.reset_index(inplace=True)
requirement_product = pd.melt(requirement_product, id_vars="sector",var_name="vehicles",value_vars=vehicles_list, value_name="value")
requirement_product.rename(columns={"value":"requirement"},inplace=True)

In [7]:
file_path = "inputs/input_market_shares.csv"
market_shares = pd.read_csv(file_path)
market_shares = pd.melt(market_shares, id_vars=["scenario","region"],var_name="vehicles",value_vars=vehicles_list, value_name="shares")
market_shares

,scenario,region,vehicles,shares
0,normal,EU,ICEV,0.213
1,normal,Non EU,ICEV,0.787
2,EU100,EU,ICEV,1.000
3,EU100,Non EU,ICEV,0.000
4,EU0,EU,ICEV,0.000
5,EU0,Non EU,ICEV,1.000
6,normal,EU,PHEV,0.213
7,normal,Non EU,PHEV,0.787
8,EU100,EU,PHEV,1.000
9,EU100,Non EU,PHEV,0.000


In [8]:
requirement_final = pd.merge(left=market_shares,right=requirement_product,on="vehicles",how="inner")
requirement_final["value"] = requirement_final["shares"]*requirement_final["requirement"]
requirement_region = requirement_final.pivot_table(index=['scenario','region','sector'],columns='vehicles',values='value',aggfunc='sum')

In [9]:
def calc_emission(vehicles):
    Y = exio3_ori.Y.copy()
    Y.iloc[:,:] = 0
    # 100% demand of EU EV
    #creating the new demand
    Y["EU",vehicles] = requirement_region.loc[scenario][vehicles]
    Y.fillna(0,inplace=True)
    exio3.Y = Y
    
    # calculate the new output required for the new demand
    exio3.x = None # reset the output
    exio3.Z = None  # expect new transaction to satisfy the new the output
    exio3.impacts.D_cba = None # reset foot print cba, others are not synch because they consider the emission from final demand
    exio3.calc_all()
    #print(exio3.meta)
    
    # calculate the satellite impact
    # calculate the impact on F
    x = exio3.x
    S = exio3.impacts.S
    diagonal_x = pd.DataFrame(0.0000000, index=x.index, columns=x.index)
    np.fill_diagonal(diagonal_x.values, x.values.flatten())
    F = S.dot(diagonal_x)
    exio3.impacts.F = F
    
    # diagonalized emission 
    diag_emission = None
    #become a matrix F of regional - sector X regional - sector 
    diag_emission = exio3.impacts.diag_stressor(stressor)
    diag_emission.F
    
    exio3.emission  = diag_emission
    exio3.calc_all()
    #print(exio3.meta)
    # now emission available in the extension exio3.emission
    
    emission = exio3.emission.D_cba
    #emission.to_csv("1_emission_"+vehicles+".csv")
    
    emission_all[vehicles]= emission.sum(axis=1)
    return

In [10]:
for scenario in scenario_list :
    emission_all = pd.DataFrame(index = exio3.Y.index, columns=vehicles_list) 
    for vehicles in vehicles_list:
        calc_emission(vehicles)
    emission_all.to_csv("outputs/emission_footprint_"+scenario+".csv")